In [31]:
import os
from collections import defaultdict

cwd = os.getcwd()
file = cwd + '/inputs/input15.txt'

data = open(file).read().splitlines()

readings = []
row = 2000000
unavailable_coords = set({})
claimed_coords = set({})

def pull_coordinates(string):
  coordinate_string = string[string.index('x'):]
  split_x_and_y = coordinate_string.split(', ')
  return { "x": int(split_x_and_y[0][2:]), "y": int(split_x_and_y[1][2:])}

def parse_line(line):
  split_sensor_and_beacon = line.split(':')
  readings.append({"sensor": pull_coordinates(split_sensor_and_beacon[0]), "beacon": pull_coordinates(split_sensor_and_beacon[1])})


def parse_data(data):
  for line in data:
    parse_line(line)

def add_coords(reading):
  sensor = reading["sensor"]
  beacon = reading["beacon"]
  beacon_distance = abs(sensor["x"] - beacon["x"]) + abs(sensor["y"] - beacon["y"])
  row_distance_y = abs(sensor["y"] - row)
  row_distance_x = beacon_distance - row_distance_y
  coord_count = 2 * (row_distance_x) + 1
  row_start = sensor["x"] - row_distance_x
  row_end = sensor["x"] + row_distance_x
  for x in range(row_end - row_start + 1):
    unavailable_coords.add(row_start + x)
  if sensor["y"] == row:
    claimed_coords.add(sensor["x"])
  if beacon["y"] == row:
    claimed_coords.add(beacon["x"])

def add_all_coords(readings):
  for reading in readings:
    add_coords(reading)

parse_data(data)
add_all_coords(readings)
solution_coords = unavailable_coords - claimed_coords
print(len(solution_coords))

4879972


In [17]:
from collections import defaultdict

cwd = os.getcwd()
file = cwd + '/inputs/input15.txt'

data = open(file).read().splitlines()
readings = []
max_coord = 4000000

def pull_coordinates(string):
  coordinate_string = string[string.index('x'):]
  split_x_and_y = coordinate_string.split(', ')
  return { "x": int(split_x_and_y[0][2:]), "y": int(split_x_and_y[1][2:])}

def parse_line(line):
  split_sensor_and_beacon = line.split(':')
  return {"sensor": pull_coordinates(split_sensor_and_beacon[0]), "beacon": pull_coordinates(split_sensor_and_beacon[1])}


def parse_data(data):
  readings = []
  for line in data:
    readings.append(parse_line(line))
  return readings

def sort_start_end(start_end):
  return start_end["start"]

def remove_contained_range(point_range):
  i = 0
  while True:
    if i+1 == len(point_range):
      return
    if point_range[i]["start"] < point_range[i+1]["start"]:
      if point_range[i]["end"] >= point_range[i+1]["end"]:
        point_range.pop(i+1)
      else:
        i += 1
    elif point_range[i]["start"] == point_range[i+1]["start"]:
      if point_range[i]["end"] >= point_range[i+1]["end"]:
        point_range.pop(i+1)
      else:
        point_range.pop(i)


def find_horizontal_gap(readings, row):
  start_ends = []
  for reading in readings:
    sensor = reading["sensor"]
    beacon = reading["beacon"]
    beacon_distance = abs(sensor["x"] - beacon["x"]) + abs(sensor["y"] - beacon["y"])
    row_distance_y = abs(sensor["y"] - row)
    if row_distance_y <= beacon_distance:
      row_distance_x = beacon_distance - row_distance_y
      start_ends.append({"start": sensor["x"] - row_distance_x, "end": sensor["x"] + row_distance_x})
  start_ends.sort(key=sort_start_end)
  remove_contained_range(start_ends)
  for i in range(len(start_ends) - 1):
    if start_ends[i+1]["start"] - start_ends[i]["end"] > 0:
      possible_gap = start_ends[i]["end"] + 1
      if possible_gap < max_coord:
        return possible_gap
  return -1

def find_tuning_frequency(readings,max_row):
  for i in range(max_row + 1):
    possible_gap = find_horizontal_gap(readings, i) 
    if possible_gap != -1:
      return(possible_gap * 4000000 + i)

readings = parse_data(data)
print(find_tuning_frequency(readings, max_coord))


56000011
